# Incremental Gramian Computation

The goal of this study is to investigate the performance effect of

1. Pre-allocating space for $Q$-matrix and Gramian $G$
2. Incrementally updating the Gramian
3. Incrementally solving for the optimal coefficient vector

in isolation. This means outside the gradient boosting framework where the substantial cost of base learner fitting obfuscates the effect. 

## Test Case

As test case, we use a simple regression setting with random true coefficient vector.

In [85]:
import numpy as np

RNG = np.random.default_rng(seed = 0)
p = 400
n = 500
beta_true = RNG.normal(size=p)
sigma_noise = 0.1
mean_x = np.zeros(p)
cov_x = np.eye(p)

x = RNG.multivariate_normal(mean_x, cov_x, size=n)
y = x.dot(beta_true) + RNG.normal(0, sigma_noise, size=n)

((y - x.dot(beta_true))**2).mean()**0.5

np.float64(0.0980956569114943)

In [86]:
def fit_naive(x, y):
    n, p = x.shape
    for j in range(p):
        gramian = x[:, :j].T.dot(x[:, :j])
        beta = np.linalg.solve(gramian, x[:, :j].T.dot(y))
    return beta

fit_naive(x, y)

array([ 1.13678756e-01, -1.16976273e-01,  5.90998512e-01,  7.16526173e-02,
       -5.60942590e-01,  4.35607072e-01,  1.34344131e+00,  9.41844608e-01,
       -6.80847076e-01, -1.21754538e+00, -5.68413359e-01,  2.48206826e-02,
       -2.33288967e+00, -2.23131795e-01, -1.18773306e+00, -8.09214921e-01,
       -4.78177414e-01, -3.05822217e-01,  4.10753219e-01,  1.07202893e+00,
       -7.96107030e-02,  1.26761115e+00, -6.10296292e-01,  3.85957251e-01,
        8.87207436e-01,  8.45539572e-02, -6.60536647e-01, -9.18526028e-01,
       -5.83762481e-01,  1.53158267e-01, -1.06282080e+00, -2.12420811e-01,
       -2.20748129e-01,  5.77478329e-01,  2.54246249e-01,  3.77525978e-01,
       -6.47405954e-01, -1.17203429e-01,  8.32826184e-01,  1.49493775e+00,
       -1.27329841e+00,  1.54224010e+00,  1.36053879e+00,  7.87691000e-01,
        2.56221638e-01, -3.38958551e-01,  1.48181271e+00,  1.95161978e+00,
        1.76313498e+00,  1.37478515e+00,  4.07455409e-01, -1.27796184e+00,
       -1.10633319e-02,  

In [87]:
def fit_naive2(x, y):
    _, p = x.shape
    for j in range(p+1):
        _x = x[:, :j]
        g = _x.T.dot(_x)
        beta = np.linalg.solve(g, _x.T.dot(y))
    return beta

((beta_true - fit_naive2(x, y))**2).mean()

np.float64(7.365294273696693e-05)

In [88]:
def fit_incremental(x, y):
    _, p = x.shape
    g =  np.zeros((p, p))
    for j in range(0, p):
        g[j, :j] = x[:, :j].T.dot(x[:, j])
        g[:j, j] = g[j, :j]
        g[j, j] = x[:, j].dot(x[:, j])
        beta = np.linalg.solve(g[:j+1, :j+1], x[:, :j+1].T.dot(y))
    return beta

np.mean((beta_true - fit_incremental(x, y))**2)

np.float64(7.365294273697835e-05)

In [89]:
%timeit fit_naive(x, y)

242 ms ± 5.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [90]:
%timeit fit_naive2(x, y)

242 ms ± 8.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
%timeit fit_incremental(x, y)

135 ms ± 3.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [94]:
from numba import njit

@njit
def numba_fit_incremental(x, y):
    _, p = x.shape
    g =  np.zeros((p, p))
    for j in range(0, p):
        g[j, :j] = x[:, :j].T.dot(x[:, j])
        g[:j, j] = g[j, :j]
        g[j, j] = x[:, j].dot(x[:, j])
        # h = g[:j+1, :j+1].copy()
        # beta = np.linalg.solve(h, x[:, :j+1].T.dot(y))
        beta = np.linalg.solve(g[:j+1, :j+1], x[:, :j+1].T.dot(y))
    return beta

x_ = np.asfortranarray(x)
np.mean((beta_true - numba_fit_incremental(x_, y))**2)

np.float64(7.365294273698925e-05)

In [95]:
# x_ = np.asfortranarray(x)
%timeit numba_fit_incremental(x_, y)

149 ms ± 9.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [96]:

@njit
def cholesky_solve_top_left(g, b, L, k):
    # Compute Cholesky: g[:k, :k] = L @ L.T
    for i in range(k):
        for j in range(i + 1):
            s = g[i, j]
            for l in range(j):
                s -= L[i, l] * L[j, l]
            if i == j:
                L[i, j] = np.sqrt(s)
            else:
                L[i, j] = s / L[j, j]

    # Solve L y = b[:k]
    y = np.empty(k)
    for i in range(k):
        s = b[i]
        for j in range(i):
            s -= L[i, j] * y[j]
        y[i] = s / L[i, i]

    # Solve L.T x = y
    x = np.empty(k)
    for i in range(k - 1, -1, -1):
        s = y[i]
        for j in range(i + 1, k):
            s -= L[j, i] * x[j]
        x[i] = s / L[i, i]

    return x

@njit
def numba_fit_incremental2(x, y):
    _, p = x.shape
    g =  np.zeros((p, p))
    l = np.zeros((p, p))
    b = np.zeros(p)
    for j in range(0, p):
        g[j, :j] = x[:, :j].T.dot(x[:, j])
        g[:j, j] = g[j, :j]
        g[j, j] = x[:, j].dot(x[:, j])

        for i in range(j+1):
            b[i] = x[:, i] @ y

        beta = cholesky_solve_top_left(g, b, l, j+1)
    return beta

np.mean((beta_true - numba_fit_incremental2(x_, y))**2)

np.float64(7.365294273696196e-05)

In [97]:
%timeit numba_fit_incremental2(x_, y)

457 ms ± 1.23 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [104]:
@njit
def cholesky_solve_top_left2(g, b, l, j):
    # Update only row j-1 in l
    row = j - 1
    for k in range(row):
        s = 0.0
        for m in range(k):
            s += l[row, m] * l[k, m]
        l[row, k] = (g[row, k] - s) / l[k, k]
    s = 0.0
    for m in range(row):
        s += l[row, m] ** 2
    l[row, row] = np.sqrt(g[row, row] - s)

    # Solve L z = b[:j]
    z = np.empty(j)
    for i in range(j):
        s = 0.0
        for k in range(i):
            s += l[i, k] * z[k]
        z[i] = (b[i] - s) / l[i, i]

    # Solve L.T beta = z
    beta = np.empty(j)
    for i in range(j - 1, -1, -1):
        s = 0.0
        for k in range(i + 1, j):
            s += l[k, i] * beta[k]
        beta[i] = (z[i] - s) / l[i, i]

    return beta

@njit
def numba_fit_incremental3(x, y):
    _, p = x.shape
    g =  np.zeros((p, p))
    l = np.zeros((p, p))
    b = np.zeros(p)
    for j in range(0, p):
        g[j, :j] = x[:, :j].T.dot(x[:, j])
        g[:j, j] = g[j, :j]
        g[j, j] = x[:, j].dot(x[:, j])

        for i in range(j+1):
            b[i] = x[:, i] @ y

        beta = cholesky_solve_top_left2(g, b, l, j+1)
    return beta

np.mean((beta_true - numba_fit_incremental3(x_, y))**2)

np.float64(7.365294273695162e-05)

In [103]:
%timeit numba_fit_incremental3(x_, y)

35.8 ms ± 200 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
